In [1]:
import numpy as np
import random
import pickle
from create_gen_0 import CreateGeneration0

In [2]:
CG0 = CreateGeneration0(number_of_fighters = 64,attribute_dimensions = 3)
CG0.pipeline()

In [3]:
import os
cwd = os.getcwd()
cwd

'C:\\Users\\Joseph\\Documents\\project_attenuation\\notebooks'

In [4]:
gen0path = os.path.abspath(os.path.join(cwd, '..', 'storage','base','data_dict_gen0.pickle'))

with open(gen0path, 'rb') as handle:
    gen0 = pickle.load(handle)


In [5]:
from alpha_motive_tournament import AlphaMotiveTournament

In [6]:
AMT = AlphaMotiveTournament(gen0)

In [7]:
AMT.tournament(printswitch=False)

In [8]:
AMT.produceAverageWinner()

In [9]:
AMT.tournament_winner_stats

array([[0.61824034, 0.08750483, 0.55689727],
       [0.61350945, 0.40339633, 0.87700837],
       [0.70488195, 0.77514184, 0.85319113]])

In [10]:
AMT.tournament_winner_stats.mean()

0.6099746144622821

In [11]:
AMT.averagewinner_stats

array([[0.55750792, 0.55855075, 0.49542559],
       [0.42079764, 0.508474  , 0.58824197],
       [0.57772797, 0.55964249, 0.53279471]])

In [12]:
AMT.averagewinner_stats.mean()

0.5332403367575336

In [13]:
AMT.resetPools()
AMT.tournament(printswitch=False)

In [14]:
AMT.resetPools()
AMT.tournament(printswitch=False)

In [15]:
AMT.fighter_dict[AMT.tournament_winner]

{'name': 'Selma Moon',
 'last_name': 'Moon',
 'alive': 0,
 'attributes': array([[0.51661846, 0.84420322, 0.29003573],
        [0.50774159, 0.51525571, 0.49289884],
        [0.31443126, 0.91951311, 0.80360177]]),
 'wins': 6,
 'tournament_count': 3}

In [16]:
gen1path = os.path.abspath(os.path.join(cwd, '..', 'storage','generation_1_pool','gen0_dead.pickle'))

with open(gen1path, 'wb') as handle:
    pickle.dump(AMT.fighter_dict,handle)

# so we have a dead generation, now lets make a new generation
first load in the old generation

In [17]:
gen0deadpath = os.path.abspath(os.path.join(cwd, '..', 'storage','generation_1_pool','gen0_dead.pickle'))

with open(gen0deadpath, 'rb') as handle:
    gen0dead = pickle.load(handle)

# create x fighters (8?) to start with, choose two random fighters and weight their values by their wins
(so f1_matrix*f1_wins*f1_mask+f2_matrix*f2_wins*f2_mask)/(f1_wins*f1_mask+f2_wins*f2_mask)

In [18]:
#grab test fighters
testfighter1 = gen0dead[list(gen0dead.keys())[0]]
testfighter2 = gen0dead[list(gen0dead.keys())[1]]
print(testfighter1,testfighter2)

#mask creating function
matrix_dimensions = testfighter1['attributes'].shape
def get_matrix_mask():
    mask_test = np.full(matrix_dimensions,True)
    mask_test[0,0] = False
    np.random.shuffle(mask_test)
    return mask_test

#(so f1_matrix*f1_wins*f1_mask+f2_matrix*f2_wins*f2_mask)/(f1_wins*f1_mask+f2_wins*f2_mask)
#get masks
tf1_mask = get_matrix_mask()
tf2_mask = get_matrix_mask()
#create pure attributes from parents
base_fighter_attributes = (testfighter1['attributes']*testfighter2['wins']*tf1_mask+testfighter2['attributes']*testfighter2['wins']*tf2_mask)/(testfighter2['wins']*tf1_mask+testfighter2['wins']*tf2_mask)
base_fighter_attributes = np.nan_to_num(base_fighter_attributes)

#set mutation rate
mutation_rate = 0.1
#mutate new fighter
mutation = np.random.rand(matrix_dimensions[0],matrix_dimensions[0])-0.5
new_fighter_attributes = base_fighter_attributes + mutation*mutation_rate
new_fighter_attributes

{'name': 'Alyda Tornak', 'last_name': 'Tornak', 'alive': 0, 'attributes': array([[0.28042826, 0.67245874, 0.10482412],
       [0.68360011, 0.7483288 , 0.53408064],
       [0.39682621, 0.99891621, 0.0513793 ]]), 'wins': 3, 'tournament_count': 3} {'name': 'Paulette Crossfire', 'last_name': 'Crossfire', 'alive': 0, 'attributes': array([[0.44557008, 0.14740741, 0.16878396],
       [0.23258242, 0.12240357, 0.72979204],
       [0.67268643, 0.43949645, 0.16173418]]), 'wins': 6, 'tournament_count': 3}


c:\users\joseph\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


array([[0.40546433, 0.42685295, 0.151353  ],
       [0.04766683, 0.44304885, 0.59529236],
       [0.55120581, 0.67728091, 0.08470546]])

### this seems to work, but the attributes are really normalised between the parents
could instead just make the probabilities that each fighter is picked based on wins, then just randomly pick atts from each

In [19]:
#grab test fighters
testfighter1 = gen0dead[list(gen0dead.keys())[0]]
testfighter2 = gen0dead[list(gen0dead.keys())[1]]
print(testfighter1,testfighter2)

#mask creating function
matrix_dimensions = testfighter1['attributes'].shape
def get_matrix_mask_2():
    mask_test = np.full(matrix_dimensions[0]**2,True)
    
    mask_test[0:int(matrix_dimensions[0]**2/2)] = False
    np.random.shuffle(mask_test)
    return mask_test.reshape(matrix_dimensions)

#(so f1_matrix*f1_wins*f1_mask+f2_matrix*f2_wins*f2_mask)/(f1_wins*f1_mask+f2_wins*f2_mask)
#get masks
tf1_mask = get_matrix_mask_2()
tf2_mask = np.invert(tf1_mask)
#create pure attributes from parents
base_fighter_attributes = (testfighter1['attributes']*tf1_mask+testfighter2['attributes']*tf2_mask)
base_fighter_attributes = np.nan_to_num(base_fighter_attributes)

#set mutation rate
mutation_rate = 0.1
#mutate new fighter
mutation = np.random.rand(matrix_dimensions[0],matrix_dimensions[0])-0.5
new_fighter_attributes = base_fighter_attributes + mutation*mutation_rate
base_fighter_attributes,new_fighter_attributes

{'name': 'Alyda Tornak', 'last_name': 'Tornak', 'alive': 0, 'attributes': array([[0.28042826, 0.67245874, 0.10482412],
       [0.68360011, 0.7483288 , 0.53408064],
       [0.39682621, 0.99891621, 0.0513793 ]]), 'wins': 3, 'tournament_count': 3} {'name': 'Paulette Crossfire', 'last_name': 'Crossfire', 'alive': 0, 'attributes': array([[0.44557008, 0.14740741, 0.16878396],
       [0.23258242, 0.12240357, 0.72979204],
       [0.67268643, 0.43949645, 0.16173418]]), 'wins': 6, 'tournament_count': 3}


(array([[0.44557008, 0.67245874, 0.10482412],
        [0.68360011, 0.12240357, 0.53408064],
        [0.67268643, 0.43949645, 0.0513793 ]]),
 array([[0.46152248, 0.68122797, 0.06665428],
        [0.69231401, 0.11232115, 0.54136037],
        [0.70834516, 0.44575092, 0.0802469 ]]))

In [20]:
np.invert(tf1_mask)

array([[ True, False, False],
       [False,  True, False],
       [ True,  True, False]])

In [21]:
tf1_mask

array([[False,  True,  True],
       [ True, False,  True],
       [False, False,  True]])

# how to choose fighters?
numpy.random.choice(indexlist, probabilities)

need to make lists with indexes and probabilities

In [22]:
indexlist = list(gen0dead.keys())
wins = []
for key in indexlist:
    wins.append(gen0dead[key]['wins'])
fighter_probabilities = np.array(wins)/sum(wins)


In [23]:
np.random.choice(indexlist, p=fighter_probabilities)

'g1_c17'

In [24]:
#tested works awesomely, lets now clean some of the cool bits up, like the name etc

In [25]:
#grab test fighters
testfighter1 = gen0dead[np.random.choice(indexlist, p=fighter_probabilities)]
testfighter2 = gen0dead[np.random.choice(indexlist, p=fighter_probabilities)]
#name new fighter
file_first_name_list = 'first-names.txt'
first_name_list_path = os.path.abspath(os.path.join(cwd, '..', 'storage','resources',file_first_name_list))
text_file = open(first_name_list_path, "r")
first_name_list = text_file.read().split('\n')
text_file.close()


print(testfighter1['name'],testfighter2['name'])
new_first_name = random.choice(first_name_list)
new_last_name = random.choice([testfighter1['last_name'],testfighter2['last_name']])
print(new_first_name,new_last_name)
#mask creating function
matrix_dimensions = testfighter1['attributes'].shape
def get_matrix_mask_2():
    mask_test = np.full(matrix_dimensions[0]**2,True)
    
    mask_test[0:int(matrix_dimensions[0]**2/2)] = False
    np.random.shuffle(mask_test)
    return mask_test.reshape(matrix_dimensions)

#(so f1_matrix*f1_wins*f1_mask+f2_matrix*f2_wins*f2_mask)/(f1_wins*f1_mask+f2_wins*f2_mask)
#get masks
tf1_mask = get_matrix_mask_2()
tf2_mask = np.invert(tf1_mask)
#create pure attributes from parents
base_fighter_attributes = (testfighter1['attributes']*tf1_mask+testfighter2['attributes']*tf2_mask)
base_fighter_attributes = np.nan_to_num(base_fighter_attributes)

#set mutation rate
mutation_rate = 0.1
#mutate new fighter
mutation = np.random.rand(matrix_dimensions[0],matrix_dimensions[0])-0.5
new_fighter_attributes = base_fighter_attributes + mutation*mutation_rate
#


new_fighter_attributes

Dari Northwind Cloris Rust
Philipa Northwind


array([[0.90874681, 0.92397261, 0.74229818],
       [0.16044533, 0.29349095, 0.06023977],
       [0.96209012, 0.89964107, 0.96224108]])